In [1]:
import pandas as pd
import os

# Read CSV file
file_path = 'London_Listings.csv'  # Replace with your file path
raw_df = pd.read_csv(file_path)

In [11]:
# Importing required libraries
import pandas as pd
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
#%matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
import time
from IPython.display import SVG
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
pd.set_option('display.max_columns', len(raw_df.columns)) # To view all columns
pd.set_option('display.max_rows', 100)

In [4]:
cols_to_drop = ['listing_url', 'scrape_id','bathrooms_text', 'last_scraped', 'name', 'description', 'neighborhood_overview','picture_url','host_id', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'calendar_last_scraped','calendar_updated']
df = raw_df.drop(cols_to_drop, axis=1)

In [5]:
df.drop(['neighbourhood_group_cleansed','license','neighbourhood'], axis=1, inplace=True)

df.set_index('id', inplace=True) # The id will be used as the index, as this could be useful in future e.g. if a separate dataset containing reviews for each property is linked to this one

In [6]:
df.drop(['host_total_listings_count', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], axis=1, inplace=True)

In [8]:
df.drop(['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'], axis=1, inplace=True)

In [9]:
# Replacing columns with f/t with 0/1
df.replace({'f': 0, 't': 1}, inplace=True)

In [10]:
df.drop(['has_availability'], axis=1, inplace=True)

In [11]:
# Converting to datetime
df.host_since = pd.to_datetime(df.host_since) 

# Calculating the number of days
df['host_days_active'] = (datetime(2024, 9, 6) - df.host_since).astype('timedelta64[D]')

# Replacing null values with the median
df.host_days_active.fillna(df.host_days_active.median(), inplace=True)

In [12]:
df.host_response_time.fillna("unknown", inplace=True)

In [13]:
# Removing the % sign from the host_response_rate string and converting to an integer
df.host_response_rate = df.host_response_rate.str[:-1].astype('float64')

In [14]:
# Bin into four categories
df.host_response_rate = pd.cut(df.host_response_rate, bins=[0, 50, 90, 99, 100], labels=['0-49%', '50-89%', '90-99%', '100%'], include_lowest=True)

# Converting to string
df.host_response_rate = df.host_response_rate.astype('str')

# Replace nulls with 'unknown'
df.host_response_rate.replace('nan', 'unknown', inplace=True)


In [15]:
# Removing the % sign from the host_response_rate string and converting to an integer
df.host_acceptance_rate = df.host_acceptance_rate.str[:-1].astype('float64')

In [16]:
# Bin into four categories
df.host_acceptance_rate = pd.cut(df.host_acceptance_rate, bins=[0, 50, 90, 99, 100], labels=['0-49%', '50-89%', '90-99%', '100%'], include_lowest=True)

# Converting to string
df.host_acceptance_rate = df.host_acceptance_rate.astype('str')

# Replace nulls with 'unknown'
df.host_acceptance_rate.replace('nan', 'unknown', inplace=True)

In [18]:
df.dropna(subset=['host_since'], inplace=True)

In [19]:
df.property_type.replace({
    'Entire rental unit': 'House',
    'Private room in rental unit': 'Apartment',
    'Entire bungalow ': 'House',
    'Entire condo': 'Apartment',
    'Entire home': 'House',
    'Private room in condo': 'Apartment',
    'Entire serviced apartment': 'Apartment',
    'Private room in townhouse': 'Apartment',
    'Entire townhouse': 'House',
    'Entire guesthouse': 'House',
    'Private room in guesthouse': 'Apartment',
    'Entire loft': 'Apartment',
    'Private room in loft': 'Apartment',
    'Private room in guest suite': 'Apartment',
    'Private room in serviced Apartment': 'Apartment',
    'Room in aparthotel': 'Apartment',
    'Room in serviced apartment': 'Apartment',
    'Private room': 'Apartment',
    'Entire vacation home': 'House',
    'Entire cottage': 'House', 
    'Entire place': 'House',
    'Tiny home': 'House',
    'Entire villa': 'House',
    'Private room in vacation home': 'Apartment',
    'Private room in cottage': 'Apartment',
    'Entire chalet ': 'House',
    }, inplace=True)

# Replacing other categories with 'other'
df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'Other'

In [20]:
for col in ['bathrooms', 'bedrooms', 'beds']:
    df[col].fillna(df[col].median(), inplace=True)

In [21]:
# Creating a set of all possible amenities
amenities_list = list(df.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.split(',')]
amenities_set = set(amenities_set)

In [22]:
df.loc[df['amenities'].str.contains('24-hour check-in'), 'check_in_24h'] = 1
df.loc[df['amenities'].str.contains('Air conditioning|Central air conditioning'), 'air_conditioning'] = 1
df.loc[df['amenities'].str.contains('Amazon Echo|Apple TV|Game console|Netflix|Projector and screen|Smart TV'), 'high_end_electronics'] = 1
df.loc[df['amenities'].str.contains('BBQ grill|Fire pit|Propane barbeque'), 'bbq'] = 1
df.loc[df['amenities'].str.contains('Balcony|Patio'), 'balcony'] = 1
df.loc[df['amenities'].str.contains('Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'), 'nature_and_views'] = 1
df.loc[df['amenities'].str.contains('Bed linens'), 'bed_linen'] = 1
df.loc[df['amenities'].str.contains('Breakfast'), 'breakfast'] = 1
df.loc[df['amenities'].str.contains('TV'), 'tv'] = 1
df.loc[df['amenities'].str.contains('Coffee maker|Espresso machine'), 'coffee_machine'] = 1
df.loc[df['amenities'].str.contains('Cooking basics'), 'cooking_basics'] = 1
df.loc[df['amenities'].str.contains('Dishwasher|Dryer|Washer'), 'white_goods'] = 1
df.loc[df['amenities'].str.contains('Elevator'), 'elevator'] = 1
df.loc[df['amenities'].str.contains('Exercise equipment|Gym|gym'), 'gym'] = 1
df.loc[df['amenities'].str.contains('Family/kid friendly|Children|children'), 'child_friendly'] = 1
df.loc[df['amenities'].str.contains('parking'), 'parking'] = 1
df.loc[df['amenities'].str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'outdoor_space'] = 1
df.loc[df['amenities'].str.contains('Host greets you'), 'host_greeting'] = 1
df.loc[df['amenities'].str.contains('Hot tub|Jetted tub|hot tub|Sauna|Pool|pool'), 'hot_tub_sauna_or_pool'] = 1
df.loc[df['amenities'].str.contains('Internet|Pocket wifi|Wifi'), 'internet'] = 1
df.loc[df['amenities'].str.contains('Long term stays allowed'), 'long_term_stays'] = 1
df.loc[df['amenities'].str.contains('Pets|pet|Cat|Cats|Dog|Dogs'), 'pets_allowed'] = 1
df.loc[df['amenities'].str.contains('Private entrance'), 'private_entrance'] = 1
df.loc[df['amenities'].str.contains('Safe|Security system'), 'secure'] = 1
df.loc[df['amenities'].str.contains('Self check-in'), 'self_check_in'] = 1
df.loc[df['amenities'].str.contains('Smoking allowed'), 'smoking_allowed'] = 1
df.loc[df['amenities'].str.contains('Step-free access|Wheelchair|Accessible'), 'accessible'] = 1
df.loc[df['amenities'].str.contains('Suitable for events'), 'event_suitable'] = 1

In [23]:
# Replacing nulls with zeros for new columns
cols_to_replace_nulls = df.iloc[:,41:].columns
df[cols_to_replace_nulls] = df[cols_to_replace_nulls].fillna(0)

# Produces a list of amenity features where one category (true or false) contains fewer than 10% of listings
infrequent_amenities = []
for col in df.iloc[:,39:].columns:
    if df[col].sum() < len(df)/10:
        infrequent_amenities.append(col)
        
# Dropping infrequent amenity features
df.drop(infrequent_amenities, axis=1, inplace=True)

# Dropping the original amenity feature
df.drop('amenities', axis=1, inplace=True)

In [24]:
df.price = df.price.str[1:-3]
df.price = df.price.str.replace(",", "")

In [25]:
df.drop(['availability_30', 'availability_60', 'availability_365'], axis=1, inplace=True)

In [26]:
df.first_review = pd.to_datetime(df.first_review) # Converting to datetime

# Calculating the number of days between the first review and the date the data was scraped
df['time_since_first_review'] = (datetime(2024, 9, 6) - df.first_review).astype('timedelta64[D]')

In [27]:
def bin_column(col, bins, labels, na_label='unknown'):
    """
    Takes in a column name, bin cut points and labels, replaces the original column with a
    binned version, and replaces nulls (with 'unknown' if unspecified).
    """
    df[col] = pd.cut(df[col], bins=bins, labels=labels, include_lowest=True)
    df[col] = df[col].cat.add_categories([na_label]).fillna(na_label)
    df[col] = df[col].astype('str')

In [28]:
# Binning time since first review
bin_column('time_since_first_review',
           bins=[0, 182, 365, 730, 1460, max(df.time_since_first_review)],
           labels=['0-6 months',
                   '6-12 months',
                   '1-2 years',
                   '2-3 years',
                   '4+ years'],
           na_label='no reviews')

In [29]:
df.last_review = pd.to_datetime(df.last_review) # Converting to datetime

# Calculating the number of days between the most recent review and the date the data was scraped
df['time_since_last_review'] = (datetime(2024, 9, 6) - df.last_review).astype('timedelta64[D]')

In [30]:
# Binning time since last review
bin_column('time_since_last_review',
           bins=[0, 14, 60, 182, 365, max(df.time_since_last_review)],
           labels=['0-2 weeks',
                   '2-8 weeks',
                   '2-6 months',
                   '6-12 months',
                   '1+ year'],
           na_label='no reviews')

In [31]:
# Dropping last_review - first_review will be kept for EDA and dropped later
df.drop('last_review', axis=1, inplace=True)

In [32]:
# Binning for all columns scored out of 10
variables_to_plot = list(df.columns[df.columns.str.startswith("review_scores") == True])

for col in variables_to_plot:
    bin_column(col,
               bins=[0, 4, 4.5, 5],
               labels=['0-4/5', '4.5/5', '5/5'],
               na_label='no reviews')

In [33]:
df.drop(['number_of_reviews_ltm', 'reviews_per_month','number_of_reviews_l30d'], axis=1, inplace=True)

In [34]:
# Dropping host_since and first_review as they are no longer needed
df.drop(['first_review'], axis=1, inplace=True)

In [35]:
from sklearn.impute import SimpleImputer

# Imputer for the most frequent value
imputer = SimpleImputer(strategy='most_frequent')

# Fit and transform the data
df['host_is_superhost'] = imputer.fit_transform(df[['host_is_superhost']])

In [37]:
# Exporteren naar CSV-bestand
df.to_csv('Preprocessed_airbnb_listings.csv', index=True)

# Controleer of het bestand correct is opgeslagen
print("Preprocessed_airbnb_listings scores zijn opgeslagen in Preprocessed_airbnb_listings.csv")

Preprocessed_airbnb_listings scores zijn opgeslagen in Preprocessed_airbnb_listings.csv
